# Use pandas to interact with database 


## Create tables using sql file¶

In [1]:
# first, make sure we have the table created in databse
import sqlite3

# create tables using the sql file thru python
# we only have to do this once. 
# create connection, if you don't already have a db it will create one
conn = sqlite3.connect('vax.db')
script = open("vaccination.sql").read()
cursor = conn.cursor()
cursor.executescript(script)
conn.close()  # make sure you close the connection


## Dump dataframe to database

Last time, we learned how to use sqlite3 natively to interact with sqlite db.

This time, we will use the functions pandas provides to help us do it.

In [2]:
import pandas as pd
import sys
sys.path.append("..")
# import the functions we need
from vaccination_project.vaccination_source import get_data_source_from_lws, content_to_df

# get the data from website and convert them to df
countries = get_data_source_from_lws(0)
country_df = content_to_df(countries)
country_df.head()

,country,iso_code,vaccines,source_name,source_website
0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Albania,ALB,Pfizer/BioNTech,Ministry of Health,https://coronavirus.al/lajme/covid19-ministria...
2,Algeria,DZA,Sputnik V,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
3,Andorra,AND,Pfizer/BioNTech,Government of Andorra,https://www.govern.ad/comunicats/item/12558-el...
4,Angola,AGO,Oxford/AstraZeneca,Ministry of Health,https://www.angop.ao/en/noticias/saude/covid-1...


In [3]:
# Now, make a connection to our database
conn = sqlite3.connect('vax.db')

# use df.to_sql method to dump dataframe to database!!
country_df.to_sql('countries', conn, if_exists='replace', index=False)
conn.close()

## Read data from data into dataframe

In [4]:
# To read data from db to dataframe directly, use pd.read_sql_qeury
conn = sqlite3.connect('vax.db')
qry = "select * from countries"
country_df = pd.read_sql_query(qry, conn)
conn.close()

In [5]:
country_df.head()

,country,iso_code,vaccines,source_name,source_website
0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Albania,ALB,Pfizer/BioNTech,Ministry of Health,https://coronavirus.al/lajme/covid19-ministria...
2,Algeria,DZA,Sputnik V,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
3,Andorra,AND,Pfizer/BioNTech,Government of Andorra,https://www.govern.ad/comunicats/item/12558-el...
4,Angola,AGO,Oxford/AstraZeneca,Ministry of Health,https://www.angop.ao/en/noticias/saude/covid-1...


## Put together helper functions

In [6]:
# Let's make helper functions so we don't forget the close the connection

from contextlib import closing
import sqlite3

def upload_df_to_db(df, db_name, table):
    with closing(sqlite3.connect(db_name))  as conn:
        df.to_sql(table, conn, if_exists='replace', index=False)
        print(f"Updated {table}!")
        
def get_data_from_db(db_name, qry):
    with closing(sqlite3.connect(db_name))  as conn:
        return pd.read_sql_query(qry, conn)


In [7]:
# see how easy this is!
get_data_from_db("vax.db", "select * from countries").head()

,country,iso_code,vaccines,source_name,source_website
0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Albania,ALB,Pfizer/BioNTech,Ministry of Health,https://coronavirus.al/lajme/covid19-ministria...
2,Algeria,DZA,Sputnik V,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
3,Andorra,AND,Pfizer/BioNTech,Government of Andorra,https://www.govern.ad/comunicats/item/12558-el...
4,Angola,AGO,Oxford/AstraZeneca,Ministry of Health,https://www.angop.ao/en/noticias/saude/covid-1...


In [8]:
# let's also upload data to vaccination_data table
vax_content = countries = get_data_source_from_lws(1)
vax_df = content_to_df(vax_content)
upload_df_to_db(vax_df, 'vax.db', 'vaccination_data')

Updated vaccination_data!


In [9]:
# now check the data
get_data_from_db('vax.db', 'select * from vaccination_data')

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
...,...,...,...,...,...,...,...,...,...,...,...
9068,Zimbabwe,2021-03-25,54892.0,54892.0,NaN,5488.0,2127.0,0.37,0.37,NaN,143.0
9069,Zimbabwe,2021-03-26,61093.0,61093.0,NaN,6201.0,2772.0,0.41,0.41,NaN,187.0
9070,Zimbabwe,2021-03-27,65466.0,65466.0,NaN,4373.0,3322.0,0.44,0.44,NaN,224.0
9071,Zimbabwe,2021-03-28,68511.0,68511.0,NaN,3045.0,3683.0,0.46,0.46,NaN,248.0


In [10]:
# let's try a join query
qry = """
SELECT * FROM vaccination_data as vd
JOIN countries as c
ON vd.country = c.country
"""

# it works!
master_df = get_data_from_db('vax.db', qry)
master_df.head()

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,country,iso_code,vaccines,source_name,source_website
0,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
